# Linear Optimization - Shipping

Shipping example

In [1]:
# Reference:  https://coin-or.github.io/pulp/

# Dependencies 
import pandas as pd
from pulp import *

In [2]:
# Parameters
country = {'Country':['Australia', 'Sweden', 'Brazil']}
factory = {'Factory':['Factory1', 'Factory2']}
products = {'Products':['Chair', 'Table', 'Bed']}
capacity = {'Factory1':500, 'Factory2': 500}


# Dataframes - normally read the data in but here I will create manually

# Demand Table
demand_data = {'Chair':[50, 120, 30], 'Table': [80, 80, 60], 'Bed':[200, 40, 175]}
data = {**country, **demand_data}
demand = pd.DataFrame(data).set_index('Country')

# Cost table
cost_data = {'Factory':['Factory1']*3 + ['Factory2']*3, 
             'Country':country['Country']*2,
             'Chair':[50, 80, 50, 80, 50, 80], 
             'Table':[60, 90, 60, 90, 60, 90], 
             'Bed':[70, 90, 70, 90, 70, 90]}

cost = pd.DataFrame(cost_data).set_index(['Factory', 'Country'])

print('Demand Table\n', demand)
print('\n', '--'*20, '\n')
print('Cost Table\n', cost)

Demand Table
            Chair  Table  Bed
Country                     
Australia     50     80  200
Sweden       120     80   40
Brazil        30     60  175

 ---------------------------------------- 

Cost Table
                     Chair  Table  Bed
Factory  Country                     
Factory1 Australia     50     60   70
         Sweden        80     90   90
         Brazil        50     60   70
Factory2 Australia     80     90   90
         Sweden        50     60   70
         Brazil        80     90   90


In [3]:
# replaces keys and cost variables from above
cost_combos={}
for product in cost.columns:
    column_dictionary = dict(cost[product])
    for dict_key, value in column_dictionary.items():
        new_key = (dict_key[0], product, dict_key[1])
        cost_combos[new_key] = value

cost_keys_pd = list(cost_combos.keys())

In [4]:
# Create linear programming model
model= LpProblem('shipping',LpMinimize)

# add model variable
var= LpVariable.dicts(name='shipment', indices=cost_keys_pd, lowBound=0, upBound=None, cat='Integer')

# name – The prefix to the name of each LP variable created
# indices – A list of strings of the keys to the dictionary of LP variables, and the main part of the variable name itself
# lowBound – The lower bound on these variables’ range. Default is negative infinity
# upBound – The upper bound on these variables’ range. Default is positive infinity
# cat – The category these variables are in, Integer or Continuous(default)
# indexs – (deprecated) Replaced with indices parameter

In [30]:
# replaces 
demand_combos={}
for product in demand.columns:
    column_dictionary = dict(demand[product])
    for dict_key, value in column_dictionary.items():
        new_key = (product, dict_key)
        demand_combos[new_key] = value

# demand_keys_pd = list(demand_combos.keys())

In [31]:
# Parameters
customer_list= country['Country']
factory_list= factory['Factory']
product_list= products['Products']

# create model constraint that sums the product of cost and product sent (aka the varying variable :: var)
model+= lpSum(var[(f,p,c)]*cost_combos[(f,p,c)]
   for f in factory_list for p in product_list for c in customer_list )

# create model constraint where the sum of the product sent is less than 500 (total capacity)
model += lpSum(var[('Factory1',p,c)]
               for p in product_list for c in customer_list)<= 500
model += lpSum(var[('Factory2',p,c)]
               for p in product_list for c in customer_list)<= 500

TypeError: list indices must be integers or slices, not str

In [7]:
# create model relationship where the product sent (var) for each product is greater than the demand
# goal is to supply all customers but remain under factory capacity
for c in customers:
    for p in products:
        model += var[('Factory1',p,c)]+var[('Factory2',p,c)]>= demand_combos[(p,c)]


In [27]:
# run the model
model.solve()

# Display results
df_results = pd.DataFrame([list(i) + [int(var[i].varValue)] for i in var], columns=['Factory', 'Product', 'Country', 'Forecast Quantity'])
df_results

,Factory,Product,Country,Forecast Quantity
0,Factory1,Chair,Australia,50
1,Factory1,Chair,Sweden,0
2,Factory1,Chair,Brazil,30
3,Factory2,Chair,Australia,0
4,Factory2,Chair,Sweden,120
5,Factory2,Chair,Brazil,0
6,Factory1,Table,Australia,80
7,Factory1,Table,Sweden,0
8,Factory1,Table,Brazil,60
9,Factory2,Table,Australia,0
